In [12]:
from dolfinx import fem, mesh, plot, io, nls
from ufl import dx, ds, grad, inner, exp, sin, diff
import ufl
from mpi4py import MPI
import petsc4py

import numpy as np

from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
import pyvista as pv
# import pyvistaqt as pvqt

import os
import shutil
import sys
import time

In [2]:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)


def logProg(sequence, every=1):

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)

    for index, record in enumerate(sequence):

        if index % every == 0:
            progress.value = index
        yield record
    progress.value = len(sequence)

In [3]:
dot = Infix(inner)


In [4]:
teta = 0.5
dt = 1E-2
lam = 1E-2
m = 1


Domain

In [5]:
domain_mesh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD, points=((0, 0), (2, 3)), n=(10, 10), cell_type=mesh.CellType.triangle
    )

base_element = ufl.FiniteElement("Lagrange", domain_mesh.ufl_cell(), 1)
space = fem.FunctionSpace(domain_mesh, base_element * base_element)

Functions

In [10]:
q, v = ufl.TestFunctions(space)

next_func = fem.Function(space)
prew_func = fem.Function(space)

c, mu = ufl.split(next_func)
c0, mu0 = ufl.split(prew_func)

# Initial
# next_func.x.array[:] = 0.0
# next_func.sub(0).interpolate(
#     lambda x: 0.63 + 0.02 *(0.5 - np.random.rand(x.shape[1]))
#     )
next_func.sub(0).interpolate(lambda x: 0.63 + 0 * x[1] + x[0])
next_func.x.scatter_forward()     # Don't know it

c = ufl.variable(c)
# f = 100 * c**2 * (1 - c)**2
f = c
dfdc = diff(f, c)

mu_mid = (1.0-teta) * mu0 + teta*mu
F0 = (c|dot|q) * dx - (c0|dot|q) * dx
+m * dt * (grad(mu_mid)|dot|grad(q)) * dx
F1 = (mu|dot|v) * dx - (dfdc|dot|v) * dx
-lam * (grad(c)|dot|grad(v)) * dx
F = F0 + F1


In [7]:
# You can't set petsc options right now for nonlinaer problem
problem = fem.petsc.NonlinearProblem(F, next_func)

solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6

ksp = solver.krylov_solver
opts = petsc4py.PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

In [19]:
try:
    shutil.rmtree('Chan-Hilliard file')
except FileNotFoundError:
    print("Directory aren't ")  


Directory are empty


In [38]:
file = io.VTKFile(MPI.COMM_WORLD, 'Chan-Hilliard file/Chan-Hilliard.xdmf', 'w')
file.write_mesh(domain_mesh)

t = 0.0
T = 50 * dt

c_func = next_func.sub(0)
c_space, c_points = space.sub(0).collapse()

prew_func.x.array[:] = next_func.x.array

for t in logProg(np.arange(0 + dt, 1, dt)):
    # result = solver.solve(next_func)
    prew_func.x.array[:] = next_func.x.array
    file.write_function(c_func, t)
    time.sleep(0.5)

file.close()

IntProgress(value=0, max=99)